In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta

from SolarCalculator.clearskycal import calsolar

In [2]:
plant = pd.read_csv('/Users/khunnoot/Desktop/year4/senior_project/solarmap_project/clear-sky-detection/plant_metadata.csv',
                   usecols = lambda column : column != 'Unnamed: 0')

In [3]:
ims30 = pd.read_csv('/Users/khunnoot/Desktop/year4/senior_project/solarmap_project/clear-sky-detection/ims/030-20220201-20230630.csv', 
                    parse_dates = ['Datetime'],
                   index_col = ['Datetime'])

In [4]:
ims30['site_name'] = 'ISL030'
ims30.index = ims30.index.tz_localize('Asia/Bangkok')
ims30 = ims30.resample('15min').last()
ims30.rename(columns={'Irradiance_1375 (W/m2)':'I'}, inplace = True)
ims30['Iclr'] = calsolar.cal_clearsky(ims30, plant.lat[29], plant.lng[29], plant.alt[29])

In [5]:
import clearsky

test = ims30.loc['2022-02-01':'2023-06-30']
len(test)

49440

In [6]:
import tools
from scipy.optimize import minimize_scalar
from scipy.linalg import hankel
import h5py

detect_GHIc = clearsky.detect_clearsky(test.I, test.Iclr, times=test.index, window_length=45,
                    mean_diff=75, max_diff=75,
                    lower_line_length=-5, upper_line_length=10,
                    var_diff=0.005, slope_dev=8, max_iterations=20,
                    return_components=False)
detect_GHIc

Datetime
2022-02-01 00:00:00+07:00    False
2022-02-01 00:15:00+07:00    False
2022-02-01 00:30:00+07:00    False
2022-02-01 00:45:00+07:00    False
2022-02-01 01:00:00+07:00    False
                             ...  
2023-06-30 22:45:00+07:00    False
2023-06-30 23:00:00+07:00    False
2023-06-30 23:15:00+07:00    False
2023-06-30 23:30:00+07:00    False
2023-06-30 23:45:00+07:00    False
Freq: 15T, Length: 49440, dtype: bool